# action queries

## 1. introduction

* action queries **modify databases**
* action queries are **not to retrieve information**, but to write new info, modify existing information or modify database structure (tables, etc.).

## 2. sample database (publications 📘)

In [1]:
# let's load jupyter sql extension

%load_ext sql
%config SqlMagic.autocommit = False

In [2]:
# load database

%sql sqlite:///data/publications.db

'Connected: @data/publications.db'

getting tables in publications database:

In [3]:
%%sql tables <<

SELECT 
    name
FROM 
    sqlite_master 
WHERE 
    type ='table' AND 
    name NOT LIKE 'sqlite_%';

 * sqlite:///data/publications.db
Done.
Returning data to local variable tables


In [4]:
tables.DataFrame()

,name
0,authors
1,discounts
2,employee
3,jobs
4,pub_info
5,publishers
6,roysched
7,sales
8,stores
9,titleauthor


## 3. types of action queries

* create / delete / modify tables
* appending / deleting / updating records in a table

### create table (as is, not as select)

```sql
CREATE TABLE [IF NOT EXISTS] [schema_name].table_name (
    column_1 data_type PRIMARY KEY,
       column_2 data_type NOT NULL,
    column_3 data_type DEFAULT 0,
    table_constraints
) [WITHOUT ROWID];
```

In [5]:
%%sql create <<

CREATE TABLE IF NOT EXISTS my_custom_table(
    my_pk_col integer primary key,
    one_column string,
    another_column datetime default '1989-07-21 01:00:00'
)

 * sqlite:///data/publications.db
Done.
Returning data to local variable create


In [6]:
create.DataFrame()  # no output for action queries!

""


### create table (as select)

In [33]:
%%sql create_as_select <<

create table if not exists store_sales_summary 
as 
select
stores.stor_id, 
stores.stor_name, 
count(distinct sales.ord_num),
count(distinct sales.title_id),
sum(sales.qty)
from sales join stores on
stores.stor_id = sales.stor_id
group by stores.stor_id, stores.stor_name;

 * sqlite:///data/publications.db
Done.
Returning data to local variable create_as_select


In [8]:
create_as_select.DataFrame()  # no output for action queries!

""


### drop table (be careful! 🚨)

be very careful, specially if you do not have a backup... how to make a backup? (production server example)...

In [32]:
%%sql drop <<

drop table if exists store_sales_summary;

 * sqlite:///data/publications.db
Done.
Returning data to local variable drop


In [20]:
drop.DataFrame()

""


### alter table add new column to table

In [15]:
%%sql alter_table_add_col <<

ALTER TABLE store_sales_summary 
ADD COLUMN goal INTEGER DEFAULT 100 NOT NULL;

 * sqlite:///data/publications.db
Done.
Returning data to local variable alter_table_add_col


In [16]:
alter_table_add_col.DataFrame()

""


### alter table drop column

not available for SQLite 😓, maybe for Postgres or MySQL...

### alter table rename column

not available for SQLite 😓, maybe for Postgres or MySQL...

### delete records in a table

In [18]:
%%sql drop_records <<

DELETE FROM store_sales_summary 
WHERE "sum(sales.qty)" < 90;

 * sqlite:///data/publications.db
3 rows affected.
Returning data to local variable drop_records


In [35]:
drop_records.DataFrame()

""


### append (insert) queries, as is (not as select)

look at the default value after executing this query... :-D

In [30]:
%%sql insert_into <<

INSERT INTO store_sales_summary 
(stor_id, 
 stor_name, 
 "count(distinct sales.ord_num)",
 "count(distinct sales.title_id)",
 "sum(sales.qty)")
VALUES (1, "my_book_store_yeah", 1000, 1000, 1000);

 * sqlite:///data/publications.db
1 rows affected.
Returning data to local variable insert_into


In [51]:
insert_into.DataFrame()

""


### append (insert) queries as select

be careful if modifications have been made to the table... rows must be equivalent, run the cell to drop and create store_sales_summary before this one...

In [31]:
%%sql insert_into_as_select <<

insert into store_sales_summary
select 
stores.stor_id, 
stores.stor_name, 
count(distinct sales.ord_num),
count(distinct sales.title_id),
sum(sales.qty)
from sales join stores on
stores.stor_id = sales.stor_id
group by stores.stor_id, stores.stor_name;

 * sqlite:///data/publications.db
(sqlite3.OperationalError) table store_sales_summary has 6 columns but 5 values were supplied
[SQL: insert into store_sales_summary select stores.stor_id, stores.stor_name, count(distinct sales.ord_num), count(distinct sales.title_id), sum(sales.qty) from sales join stores on stores.stor_id = sales.stor_id group by stores.stor_id, stores.stor_name;]
(Background on this error at: http://sqlalche.me/e/e3q8)


### update rows

run the cell to create goal column before this one!

In [58]:
%%sql update <<

update store_sales_summary
set goal = goal + 50;

 * sqlite:///data/publications.db
12 rows affected.
Returning data to local variable update


In [60]:
update.DataFrame()

""


### update rows conditionally

In [63]:
%%sql update_conditional <<

update store_sales_summary
set goal = goal + 50
where "count(distinct sales.ord_num)" < 4;

 * sqlite:///data/publications.db
10 rows affected.
Returning data to local variable update_conditional


In [64]:
update_conditional.DataFrame()

""


## 4. bonus: operate with sqlalchemy over databases

In [65]:
from sqlalchemy import create_engine

In [66]:
eng = create_engine('sqlite:///data/publications.db')

In [72]:
eng.execute("select * from sales;").fetchone()

('6380', '6871', '1994-09-14 00:00:00', 5, 'Net 60', 'BU1032')

https://docs.sqlalchemy.org/en/13/core/connections.html

### 5. transactional vs analytical databases